# Setup

In [1]:
%load_ext autoreload
%autoreload 2

## Install packages

In [ ]:
! pip install terratorch==1.0.1 gdown tensorboard

In [3]:
# Run this line if you are using Google Colab
# ! wget https://raw.githubusercontent.com/oroikono/IBM-BlueSky-Challenge-ZeroFlood/main/dataset.py
# ! wget https://raw.githubusercontent.com/oroikono/IBM-BlueSky-Challenge-ZeroFlood/main/model.py
# ! wget https://raw.githubusercontent.com/oroikono/IBM-BlueSky-Challenge-ZeroFlood/main/tools.py

## Import modules

In [2]:
import os
import torch

## Download the dataset

In [5]:
! git lfs install
! git clone https://huggingface.co/datasets/hk-kaden-kim/BlueSky-Challenge-Dataset

Git LFS initialized.
Cloning into 'BlueSky-Challenge-Dataset'...
remote: Enumerating objects: 13271, done.
remote: Total 13271 (delta 0), reused 0 (delta 0), pack-reused 13271 (from 1)
Receiving objects: 100% (13271/13271), 7.92 MiB | 17.78 MiB/s, done.
Resolving deltas: 100% (6003/6003), done.
Updating files: 100% (13297/13297), done.
Filtering content: 100% (6645/6645), 2.31 GiB | 33.03 MiB/s, done.


In [6]:
# Dataset statistics
DATASET_DIR = "BlueSky-Challenge-Dataset"
for split in ["train", "test", "val"]:
  png_files_cnt = 0
  csv_files_cnt = 0
  for root, dirs, files in os.walk(os.path.join(DATASET_DIR, split)):
      for file in files:
          if file.endswith('.png'):
              png_files_cnt += 1
          if file.endswith('.csv'):
              csv_files_cnt += 1
  print(f"{split}: {png_files_cnt} images and {csv_files_cnt} masks")

train: 3184 images and 3184 masks
test: 1332 images and 1332 masks
val: 2128 images and 2128 masks


# Training

In [3]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl

from dataset import FloodDataModule
from model import FloodRiskModel

In [4]:
MAX_EPOCHS = 1 # 10
BATCH_SIZE = 4 # 16
TiM = ["S1RTC","DEM","LULC","NDVI"]

In [7]:
# Initialize datamodule
datamodule = FloodDataModule(
    local_path=DATASET_DIR,
    batch_size=BATCH_SIZE,  # Adjust based on GPU memory
    num_workers=8
)
datamodule.setup("fit")
sample = datamodule.train_dataset[0]

print(f"\nDataset Summary:")
print(f"Training samples: {len(datamodule.train_dataset)}")
print(f"Validation samples: {len(datamodule.val_dataset)}")
print(f"Sample keys: {sample.keys()}")
print(f"Image shape: {sample['image'].shape}")
print(f"Mask shape: {sample['mask'].shape}")
print(f"Mask unique values: {torch.unique(sample['mask'])}")
print()

Training samples: 3184
Validation samples: 2128

Dataset Summary:
Training samples: 3184
Validation samples: 2128
Sample keys: dict_keys(['image', 'mask'])
Image shape: torch.Size([3, 256, 256])
Mask shape: torch.Size([256, 256])
Mask unique values: tensor([0, 1])



In [8]:
# Initialize model
model = FloodRiskModel(num_classes=2, tim=TiM)
assert isinstance(model, pl.LightningModule)

# Setup callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="output/flood_risk/checkpoints/",
    filename="best-flood-iou-{epoch}-{val_iou:.3f}",
    monitor="val_iou",
    mode="max",
    save_top_k=3,
    save_last=True
)

logger = TensorBoardLogger("output/flood_risk/logs", name="flood_risk")

# Setup trainer
trainer = pl.Trainer(
    max_epochs=MAX_EPOCHS,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    callbacks=[checkpoint_callback],
    logger=logger,
    log_every_n_steps=100,
    gradient_clip_val=1.0,
    enable_checkpointing=True,
    detect_anomaly=False,  # Disable anomaly detection to save memory
)

print(" Starting flood segmentation training with CSV masks...")
trainer.fit(model, datamodule=datamodule)
print(" Training completed!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


 Starting flood segmentation training with CSV masks...
Training samples: 3184


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation samples: 2128


INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                     | Params | Mode 
---------------------------------------------------------------
0 | model     | SemanticSegmentationTask | 416 M  | train
1 | train_iou | MulticlassJaccardIndex   | 0      | train
2 | val_iou   | MulticlassJaccardIndex   | 0      | train
3 | train_acc | MulticlassAccuracy       | 0      | train
4 | val_acc   | MulticlassAccuracy       | 0      | train
5 | train_f1  | MulticlassF1Score        | 0      | train
6 | val_f1    | MulticlassF1Score        | 0      | train
---------------------------------------------------------------
139 M     Trainable params
276 M     Non-trainable params
416 M     Total params
1,666.976 Total estimated model params size (MB)
759       Modules in train mode
0         Modules in eval mode


...

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


 Training completed!
